<a href="https://colab.research.google.com/github/sachinmathewjose/movie-recomendation-DL/blob/master/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#run this in the first time
!mkdir logs
!cp -r '/content/drive/My Drive/colab/recomendation/datset/.' '/content/'
!tar -xvf nf_prize_dataset.tar.gz
!mkdir data
!tar -C /content/data -xf download/training_set.tar
!rm -r download
!mkdir output
!mkdir netflix_data

download/
download/rmse.pl
download/README
download/qualifying.txt
download/probe.txt
download/movie_titles.txt
download/training_set.tar


###Preprocessing the data

In [0]:
from os import listdir, path, makedirs
import random
import sys
import time
import datetime
import csv
import keras # imports keras and tensorflow as backend
import matplotlib.pyplot as plt # imports matplotlib
import numpy as np
import pandas as pd
%matplotlib inline
import os
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import TensorBoard
from keras import optimizers
import keras.backend as K

Using TensorFlow backend.


In [0]:
def print_stats(data):
  total_ratings = 0
  print("STATS")
  for user in data:
    total_ratings += len(data[user])
  print("Total Ratings: {}".format(total_ratings))
  print("Total User count: {}".format(len(data.keys())))
  
  
def save_data_to_file(data, filename):
  with open(filename, 'w') as out:
    for userId in data:
      for record in data[userId]:
        out.write("{}\t{}\t{}\n".format(userId, record[0], record[1]))

def create_NETFLIX_data_timesplit(all_data,
                                  train_min,
                                  train_max,
                                  test_min,
                                  test_max):
  """
  Creates time-based split of NETFLIX data into train, and (validation, test)
  :param all_data:
  :param train_min:
  :param train_max:
  :param test_min:
  :param test_max:
  :return:
  """
  train_min_ts = time.mktime(datetime.datetime.strptime(train_min,"%Y-%m-%d").timetuple())
  train_max_ts = time.mktime(datetime.datetime.strptime(train_max, "%Y-%m-%d").timetuple())
  test_min_ts = time.mktime(datetime.datetime.strptime(test_min, "%Y-%m-%d").timetuple())
  test_max_ts = time.mktime(datetime.datetime.strptime(test_max, "%Y-%m-%d").timetuple())

  training_data = dict()
  validation_data = dict()
  test_data = dict()

  train_set_items = set()

  for userId, userRatings in all_data.items():
    time_sorted_ratings = sorted(userRatings, key=lambda x: x[2])  # sort by timestamp
    for rating_item in time_sorted_ratings:
      if rating_item[2] >= train_min_ts and rating_item[2] <= train_max_ts:
        if not userId in training_data:
          training_data[userId] = []
        training_data[userId].append(rating_item)
        train_set_items.add(rating_item[0]) # keep track of items from training set
      elif rating_item[2] >= test_min_ts and rating_item[2] <= test_max_ts:
        if not userId in training_data: # only include users seen in the training set
          continue
        p = random.random()
        if p <=0.5:
          if not userId in validation_data:
            validation_data[userId] = []
          validation_data[userId].append(rating_item)
        else:
          if not userId in test_data:
            test_data[userId] = []
          test_data[userId].append(rating_item)

  # remove items not not seen in training set
  for userId, userRatings in test_data.items():
    test_data[userId] = [rating for rating in userRatings if rating[0] in train_set_items]
  for userId, userRatings in validation_data.items():
    validation_data[userId] = [rating for rating in userRatings if rating[0] in train_set_items]

  return training_data, validation_data, test_data

In [0]:

user2id_map = dict()
item2id_map = dict()
userId = 0
itemId = 0
all_data = dict()

folder = 'data/training_set/'
out_folder = 'netflix_data'
# create necessary folders:
for output_dir in [(out_folder + f) for f in [
  "/N6M_TRAIN", "/N6M_VALID", "/N6M_TEST"]]:
  makedirs(output_dir, exist_ok=True)

text_files = [path.join(folder, f)
              for f in listdir(folder)
              if path.isfile(path.join(folder, f)) and ('.txt' in f)]
i = 0
for text_file in text_files:
  i = i+1
  if i>11000:
    with open('netflix_data/dict.csv', 'w') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in all_data.items():
        writer.writerow([key, value], delimiter=':')
    all_data.clear()
  with open(text_file, 'r') as f:
    print("Processing: {}".format(text_file))
    lines = f.readlines()
    item = int(lines[0][:-2]) # remove newline and :
    if not item in item2id_map:
      item2id_map[item] = itemId
      itemId += 1

    for rating in lines[1:]:
      parts = rating.strip().split(",")
      user = int(parts[0])
      if not user in user2id_map:
        user2id_map[user] = userId
        userId += 1
      rating = float(parts[1])
      ts = int(time.mktime(datetime.datetime.strptime(parts[2],"%Y-%m-%d").timetuple()))
      if user2id_map[user] not in all_data:
        all_data[user2id_map[user]] = []
      all_data[user2id_map[user]].append((item2id_map[item], rating, ts))

with open('netflix_data/dict2.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  for key, value in all_data.items():
    writer.writerow([key, value])


In [0]:
(n6m_train, n6m_valid, n6m_test) = create_NETFLIX_data_timesplit(all_data,
                                                                 "2005-06-01",
                                                                 "2005-11-30",
                                                                 "2005-12-01",
                                                                 "2005-12-31")
print("Netflix 6m train")
print_stats(n6m_train)
save_data_to_file(n6m_train, out_folder+"/N6M_TRAIN/n6m.train.txt")
print("Netflix 6m valid")
print_stats(n6m_valid)
save_data_to_file(n6m_valid, out_folder + "/N6M_VALID/n6m.valid.txt")
print("Netflix 6m test")
print_stats(n6m_test)
save_data_to_file(n6m_test, out_folder + "/N6M_TEST/n6m.test.txt")

In [0]:
!cp -r netflix_data '/content/drive/My Drive/colab/recomendation/'


###Now the data is available in drive

In [0]:
!cp -r '/content/drive/My Drive/colab/recomendation/netflix_data' .

In [0]:
NAME = "DeepRecommender-DR-BaselineLR-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='./logs/{}'.format(NAME))
DATA_DIR = 'netflix_data'
nf_6m_train = os.path.join(DATA_DIR, 'N6M_TRAIN', 'n6m.train.txt')
nf_6m_valid = os.path.join(DATA_DIR, 'N6M_VALID', 'n6m.valid.txt')
nf_6m_test = os.path.join(DATA_DIR, 'N6M_TEST', 'n6m.test.txt')

In [0]:
df_train = pd.read_csv(nf_3m_train, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_train.shape)
df_train.head()
df_valid = pd.read_csv(nf_3m_valid, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_valid.shape)
df_valid.head()
df_test = pd.read_csv(nf_3m_test, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_test.shape)
df_test.head()

In [0]:
customer_map = df_train.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[0:17550]
#user_map
num_users = len(customer_map)
customer_map[-1]

movie_map = df_train.MovieID.unique()
movie_map.sort()
movie_map = movie_map[0:1000]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

In [0]:
#run this only once
file = open('matrix_train.csv','w')
def temp(i):
    return np.where(movie_map == i)[0][0]

for id_user in customer_map:
    id_movie = df_train.iloc[:,1][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
    id_movie = id_movie.apply(temp)
    id_rating=df_train.iloc[:,2][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
    ratings=np.zeros(num_movies, dtype=np.uint32)
    ratings[id_movie-1]=id_rating
    if sum(ratings)==0:
        continue
    ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
    file.write(ratings.to_csv(index=False, header=False))
    del id_movie
    del id_rating
    del ratings


file.close()


X = pd.read_csv('matrix_train.csv', header=None)
print(X.shape)
X.head()

In [0]:
customer_map = df_valid.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

movie_map = df_valid.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:895]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

In [0]:
file = open('matrix_valid.csv','w')

def temp(i):
    return np.where(movie_map == i)[0][0]

for id_user in customer_map:
    id_movie = df_valid.iloc[:,1][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
    id_movie = id_movie.apply(temp)
    id_rating=df_valid.iloc[:,2][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
    ratings=np.zeros(1000, dtype=np.uint32)
    ratings[id_movie-1]=id_rating
    if sum(ratings)==0:
        continue
    ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
    file.write(ratings.to_csv(index=False, header=False))
    del id_movie
    del id_rating
    del ratings

file.close()


X_valid = pd.read_csv('matrix_valid.csv', header=None)
print(X_valid.shape)
X_valid.head()

In [0]:
customer_map = df_test.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

movie_map = df_test.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:901]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

In [0]:
file = open('matrix_test.csv','w')

def temp(i):
    return np.where(movie_map == i)[0][0]

for id_user in customer_map:
    id_movie = df_test.iloc[:,1][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
    id_movie = id_movie.apply(temp)
    id_rating=df_test.iloc[:,2][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
    ratings=np.zeros(1000, dtype=np.uint32)
    ratings[id_movie-1]=id_rating
    if sum(ratings)==0:
        continue
    ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
    file.write(ratings.to_csv(index=False, header=False))
    del id_movie
    del id_rating
    del ratings

file.close()

X_test = pd.read_csv('matrix_test.csv', header=None)
print(X_test.shape)
X_test.head()

In [0]:
def rmse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    masked_squared_error = mask_true * K.square((y_true - y_pred))
    # in case mask_true is 0 everywhere, the error would be nan, therefore divide by at least 1
    # this doesn't change anything as where sum(mask_true)==0, sum(masked_squared_error)==0 as well
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1)
    return K.sqrt(masked_mse)

In [0]:
model = Sequential()

model.add(Dense(28, input_dim = X.shape[1], activation='selu'))
model.add(Dense(56, activation='selu'))
model.add(Dense(56, activation='selu'))
model.add(Dropout(0.65))
model.add(Dense(56, activation='selu'))
model.add(Dense(28, activation='selu'))
model.add(Dense(X.shape[1], activation='selu'))

In [0]:
sgd = optimizers.SGD(lr=0.005, momentum=0.9)
model.compile(loss=rmse, optimizer=sgd)

model.fit(X, X, batch_size=128, epochs=100, validation_data=(X_valid, X_valid), callbacks=[tensorboard])

In [0]:
test_loss = model.evaluate(X_test, X_test)
test_loss